# Simulation studies using PAISA data

In [1]:
using CSV
using DataFrames
using DelimitedFiles

# helper function to submit 1 job to run 1 command
function submit(command::String, ncores::Int, total_mem::Number, 
        joblog_dir::String="/oak/stanford/groups/zihuai/paisa/slurms"; 
        jobname="submit", waitfor=Int[], verbose=true, highp=false)
    mem = round(Int, total_mem / ncores) # memory per core
    filename = "$jobname.sh"
    open(filename, "w") do io
        println(io, "#!/bin/bash")
        println(io, "#")
        println(io, "#SBATCH --job-name=$jobname")
        println(io, "#")
        if highp
            println(io, "#SBATCH --time=168:00:00")
        else
            println(io, "#SBATCH --time=24:00:00")
        end
        println(io, "#SBATCH --cpus-per-task=$ncores")
        println(io, "#SBATCH --mem-per-cpu=$(mem)G")
        if highp
            println(io, "#SBATCH --partition=candes,zihuai")
        else
            println(io, "#SBATCH --partition=candes,zihuai,normal,owners")
        end
        println(io, "#SBATCH --output=$(joinpath(joblog_dir, "slurm-%j.out"))")
        println(io, "")
        println(io, "#save job info on joblog:")
        println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
        println(io, "")
        println(io, "# load the job environment:")
#         println(io, "module load julia/1.9.4")
#         println(io, "module load biology plink/1.90b5.3")
#         println(io, "module load R/4.0.2")
#         println(io, "export OPENBLAS_NUM_THREADS=1")
        println(io, "export JULIA_DEPOT_PATH=\"/home/groups/sabatti/.julia\"")
        println(io, "")
        println(io, "# run code")
        println(io, "echo \"$command\"")
        println(io, "$command")
        println(io, "")
        println(io, "#echo job info on joblog:")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
        println(io, "#echo \" \"")
    end
    # submit job and capture job ID
    io = IOBuffer()
    if length(waitfor) != 0
        run(pipeline(`sbatch --dependency=afterok:$(join(waitfor, ':')) $filename`; stdout=io))
    else
        run(pipeline(`sbatch $filename`; stdout=io))
    end
    msg = String(take!(io))
    verbose && print(stdout, msg)
    jobid = parse(Int, strip(msg)[21:end])
    # clean up and return job ID
    close(io)
    rm(filename, force=true)
    return jobid
end

# helper function to submit 1 job to run multiple commands
function submit(commands::Vector{String}, ncores::Int, total_mem::Number, 
        joblog_dir::String="/oak/stanford/groups/zihuai/paisa/slurms"; 
        jobname="submit", waitfor=Int[], verbose=true, highp=false)
    mem = round(Int, total_mem / ncores) # memory per core
    filename = "$jobname.sh"
    open(filename, "w") do io
        println(io, "#!/bin/bash")
        println(io, "#")
        println(io, "#SBATCH --job-name=$jobname")
        println(io, "#")
        if highp
            println(io, "#SBATCH --time=168:00:00")
        else
            println(io, "#SBATCH --time=24:00:00")
        end
        println(io, "#SBATCH --cpus-per-task=$ncores")
        println(io, "#SBATCH --mem-per-cpu=$(mem)G")
        if highp
            println(io, "#SBATCH --partition=candes,zihuai")
        else
            println(io, "#SBATCH --partition=candes,zihuai,normal,owners")
        end
        println(io, "#SBATCH --output=$(joinpath(joblog_dir, "slurm-%j.out"))")
        println(io, "")
        println(io, "#save job info on joblog:")
        println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
        println(io, "")
        println(io, "# load the job environment:")
#         println(io, "module load julia/1.9.4")
#         println(io, "module load biology plink/1.90b5.3")
#         println(io, "module load R/4.0.2")
#         println(io, "export OPENBLAS_NUM_THREADS=1")
        println(io, "export JULIA_DEPOT_PATH=\"/home/groups/sabatti/.julia\"")
        println(io, "")
        for command in commands
            println(io, "echo \"$command\"")
            println(io, "$command")
        end
        println(io, "")
        println(io, "#echo job info on joblog:")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
        println(io, "#echo \" \"")
    end
    # submit job and capture job ID
    io = IOBuffer()
    if length(waitfor) != 0
        run(pipeline(`sbatch --dependency=afterok:$(join(waitfor, ':')) $filename`; stdout=io))
    else
        run(pipeline(`sbatch $filename`; stdout=io))
    end
    msg = String(take!(io))
    verbose && print(stdout, msg)
    jobid = parse(Int, strip(msg)[21:end])
    # clean up and return job ID
    close(io)
    rm(filename, force=true)
    return jobid
end


"Run a Cmd object, returning the stdout & stderr contents plus the exit code"
function execute(cmd::Cmd)
    out = Pipe()
    err = Pipe()

    process = run(pipeline(ignorestatus(cmd), stdout=out, stderr=err))
    close(out.in)
    close(err.in)

    return (
        stdout = String(read(out)), 
        stderr = String(read(err)),  
        code = process.exitcode
    )
end

function get_job_names()
    data_str, _, _ = execute(`squeue -u bbchu -h -o "%.30j"`)
    lines = split(data_str, "\n")
    jobnames = String[]
    for line in lines
        push!(jobnames, strip(line))
    end
    return jobnames
end

[ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
[ Info: Precompiling DelimitedFiles [8bb1440f-4735-579b-a4ab-409b98df4dab]


get_job_names (generic function with 1 method)

## Run simulations

This simulation: 
+ randomly chooses $k=50$ causal SNPs in chr21
+ Z scores are formed by $z = X'y / sqrt(N)$
+ marginal p-values computed by transforming Z-scores

**This simulation doesn't quite work.** Knockoffs have power worse than marginal studies and FDR doesn't seem to be controlled, although the marginal false positive rate is much worse. 

In [ ]:
# put in script /oak/stanford/groups/zihuai/paisa/power_sim.jl

using SnpArrays, Random, CSV, DataFrames, Distributions, StatsBase, GhostKnockoffGWAS
include("/oak/stanford/groups/zihuai/paisa/utilities.jl")

seed = parse(Int, ARGS[1])
LD_files = ARGS[2]
datadir = ARGS[3] 
chr = parse(Int, ARGS[4])

# for testing
# seed = 1111 # computes shrinkage on all SNPs
# seed = 1112 # computes shrinkage on reps
# LD_files = "/oak/stanford/groups/zihuai/paisa/LD_files_large2"
# datadir = "/oak/stanford/groups/zihuai/paisa/GLIMPSE2_maf0.05"
# chr = 22

# some helper functions
function pval2zscore(pvals::AbstractVector{T}, beta::AbstractVector{T}) where T
    length(pvals) == length(beta) || 
        error("pval2zscore: pvals and beta should have the same length")
    return zscore.(pvals, beta)
end
zscore(p::T, beta::T) where T = sign(beta) * quantile(Normal(), p/2)
pval(z::T) where T = 2ccdf(Normal(), abs(z))

# some unrealistic simulation parameters I made up just to check things works
k = 50
mu = 0
sigma = 0.1 # beta ~ N(mu, sigma)

# import genotypes
xdata = SnpData(joinpath(datadir, "chr$chr"))
X = SnpLinAlg{Float64}(xdata.snparray, center=true, scale=true, impute=true)
n, p = size(X)

# simulate phenotypes and normalize it
Random.seed!(seed)
beta = zeros(p)
beta[1:k] .= rand(Normal(mu, sigma), k)
shuffle!(beta)
y = X * beta + randn(n)
zscore!(y, mean(y), std(y))

# marginal association test: Z scores and associated p-values
z = X'*y ./ sqrt(n)
pvals = pval.(z)

# run GhostKnockoffGWAS on output of `solveblock`
chrs = parse.(Int, xdata.snp_info.chromosome)
pos = xdata.snp_info.position
ref = xdata.snp_info.allele1
alt = xdata.snp_info.allele2
hg_build = 38
outdir = "/oak/stanford/groups/zihuai/paisa/sims"
outname = "sim$seed"
total_time = @elapsed GhostKnockoffGWAS.ghostknockoffgwas(
    LD_files, z, chrs, pos, Vector(ref), Vector(alt), n, hg_build, outdir,
    outname = outname)

# compute power
outfile = joinpath(outdir, outname)
snps = SNPs(xdata.snp_info.chromosome, pos, ref, alt)
causal_snps = snps[findall(!iszero, beta)]
GK_df = CSV.read(outfile * ".txt", DataFrame)
GK_snps = SNPs(string.(GK_df.chr), GK_df.pos_hg38, GK_df.ref, GK_df.alt)
GK_df[!, "true_beta"] = zeros(size(GK_df, 1))
idx1 = filter!(!isnothing, indexin(causal_snps, GK_snps))
idx2 = filter!(!isnothing, indexin(causal_snps, snps))
GK_df[idx1, "true_beta"] .= beta[idx2]
GK_discover = findall(isone, GK_df[!, "selected_fdr0.1"])
GK_snps = SNPs(string.(GK_df[!, "chr"]), GK_df[!, "pos_hg38"], GK_df[!, "ref"], GK_df[!, "alt"])
GK_discover_snps = GK_snps[GK_discover]
GK_power = length(intersect(GK_discover_snps, causal_snps)) / length(causal_snps)
GK_FP = length(setdiff(GK_discover_snps, causal_snps))

# marginal power (use pvalue from GK_df since some SNPs are deleted)
# marginal_discover_snps = snps[findall(x -> x < 0.05/length(beta), pvals)]
# marginal_power = length(intersect(marginal_discover_snps, causal_snps)) / length(causal_snps)
# marginal_FP = length(setdiff(marginal_discover_snps, causal_snps))
marginal_pvals = GK_df[!, "pvals"]
marginal_discoveries = findall(x -> x < 0.05/size(GK_df, 1), marginal_pvals)
true_beta_idx = findall(!iszero, GK_df[!, "true_beta"])
marginal_power = length(intersect(marginal_discoveries, true_beta_idx)) / length(causal_snps)
marginal_FP = length(setdiff(marginal_discoveries, true_beta_idx))
println("\n\n marginal_power = $marginal_power, marginal false positives = $marginal_FP")
println("GK_power = $GK_power, GK false positives = $GK_FP")
println("Total GK time = $total_time")

# save GK_df with causal betas appended
CSV.write(outfile * ".txt", GK_df)

# write output
outfile = "/oak/stanford/groups/zihuai/paisa/sims/sim$(seed)_result.txt"
open(outfile, "w") do io
    println(io, "GK_power,$GK_power")
    println(io, "marginal_power,$marginal_power")
    println(io, "GK_FP,$GK_FP")
    println(io, "marginal_FP,$marginal_FP")
    println(io, "total_time,$total_time")
end

println("\nDone!")

Submit jobs (MAF > 0.01)

In [2]:
exe = "/oak/stanford/groups/zihuai/paisa/power_sim.jl"
LD_files = "/oak/stanford/groups/zihuai/paisa/LD_files"
datadir = "/oak/stanford/groups/zihuai/paisa/GLIMPSE2_maf0.01"
chr = 21
for sim in 1:10
    submit("julia $exe $sim $LD_files $datadir $chr", 1, 64, jobname="sim$sim")
end

Submitted batch job 51959063
Submitted batch job 51959064
Submitted batch job 51959065
Submitted batch job 51959066
Submitted batch job 51959067
Submitted batch job 51959068
Submitted batch job 51959069
Submitted batch job 51959070
Submitted batch job 51959071
Submitted batch job 51959072


## Simulation 2

This simulation copies Zihuai's simulation:

+ Simulation is run genome-wide
+ Randomly choose $k$ regions
+ Each region randomly pick 1 causal SNP

```julia
# merge PLINK files by chr if not done already
# cd /oak/stanford/groups/zihuai/paisa/GLIMPSE2_maf0.05
using SnpArrays
prefix = "chr"
merge_plink(prefix; des = "allchr")
```

In [ ]:
# put in script /oak/stanford/groups/zihuai/paisa/power_sim.jl

using SnpArrays, Random, CSV, DataFrames, Distributions, StatsBase, GhostKnockoffGWAS
include("/oak/stanford/groups/zihuai/paisa/utilities.jl")

seed = parse(Int, ARGS[1])
LD_files = ARGS[2]
datadir = ARGS[3] 

# for testing
# seed = 1111
# LD_files = "/oak/stanford/groups/zihuai/paisa/LD_files"
# datadir = "/oak/stanford/groups/zihuai/paisa/GLIMPSE2_maf0.01"

# some helper functions
function pval2zscore(pvals::AbstractVector{T}, beta::AbstractVector{T}) where T
    length(pvals) == length(beta) || 
        error("pval2zscore: pvals and beta should have the same length")
    return zscore.(pvals, beta)
end
zscore(p::T, beta::T) where T = sign(beta) * quantile(Normal(), p/2)
pval(z::T) where T = 2ccdf(Normal(), abs(z))

# simulation parameters
k = 50

# import genotypes
xdata = SnpData(joinpath(datadir, "allchr"))
X = SnpLinAlg{Float64}(xdata.snparray, center=true, scale=true, impute=true)
n, p = size(xdata)

# randomly pick k regions to have 1 causal SNP
Random.seed!(seed)
beta = zeros(p)
regions_df = CSV.read(joinpath(LD_files, "all_regions.txt"), DataFrame)
causal_regions = sample(1:size(regions_df, 1), k, replace=false)
for i in causal_regions
    try
        chr, start_pos, end_pos = regions_df[i, :]
        info_file = joinpath(LD_files, "chr$chr", "Info_start$(start_pos)_end$(end_pos).csv")
        info = CSV.read(info_file, DataFrame)
        pi = size(info, 1)
        _, AF, causal_chr, causal_pos, causal_ref, causal_alt = info[sample(1:pi), :]
        MAF = AF > 0.5 ? 1 - AF : AF
        idx = findfirst(x -> 
            x.chromosome == string(causal_chr) && 
            x.position == causal_pos && 
            x.allele1 == causal_ref && 
            x.allele2 == causal_alt, 
            eachrow(xdata.snp_info)
        )
        beta[idx] = 1 / sqrt(2MAF * (1 - MAF))
    catch
        continue
    end
end

# simulate phenotypes and normalize it
Random.seed!(seed)
y = X * beta + rand(Normal(0, sqrt(3)), n) # ~100 sec
zscore!(y, mean(y), std(y))

# marginal association test: Z scores and associated p-values
z = X'*y ./ sqrt(n)
pvals = pval.(z)

# run GhostKnockoffGWAS on output of `solveblock`
chrs = parse.(Int, xdata.snp_info.chromosome)
pos = xdata.snp_info.position
ref = xdata.snp_info.allele1
alt = xdata.snp_info.allele2
hg_build = 38
outdir = "/oak/stanford/groups/zihuai/paisa/sims"
outname = "sim$seed"
total_time = @elapsed GhostKnockoffGWAS.ghostknockoffgwas(
    LD_files, z, chrs, pos, Vector(ref), Vector(alt), n, hg_build, outdir,
    outname = outname)

# compute power
outfile = joinpath(outdir, outname)
snps = SNPs(xdata.snp_info.chromosome, pos, ref, alt)
causal_snps = snps[findall(!iszero, beta)]
GK_df = CSV.read(outfile * ".txt", DataFrame)
GK_snps = SNPs(string.(GK_df.chr), GK_df.pos_hg38, GK_df.ref, GK_df.alt)
GK_df[!, "true_beta"] = zeros(size(GK_df, 1))
idx1 = filter!(!isnothing, indexin(causal_snps, GK_snps))
idx2 = filter!(!isnothing, indexin(causal_snps, snps))
GK_df[idx1, "true_beta"] .= beta[idx2]
GK_discover = findall(isone, GK_df[!, "selected_fdr0.1"])
GK_snps = SNPs(string.(GK_df[!, "chr"]), GK_df[!, "pos_hg38"], GK_df[!, "ref"], GK_df[!, "alt"])
GK_discover_snps = GK_snps[GK_discover]
GK_power = length(intersect(GK_discover_snps, causal_snps)) / length(causal_snps)
GK_FP = length(setdiff(GK_discover_snps, causal_snps))

# marginal power (use pvalue from GK_df since some SNPs are deleted)
# marginal_discover_snps = snps[findall(x -> x < 0.05/length(beta), pvals)]
# marginal_power = length(intersect(marginal_discover_snps, causal_snps)) / length(causal_snps)
# marginal_FP = length(setdiff(marginal_discover_snps, causal_snps))
marginal_pvals = GK_df[!, "pvals"]
marginal_discoveries = findall(x -> x < 5e-8, marginal_pvals)
true_beta_idx = findall(!iszero, GK_df[!, "true_beta"])
marginal_power = length(intersect(marginal_discoveries, true_beta_idx)) / length(causal_snps)
marginal_FP = length(setdiff(marginal_discoveries, true_beta_idx))
println("\n\n marginal_power = $marginal_power, marginal false positives = $marginal_FP")
println("GK_power = $GK_power, GK false positives = $GK_FP")
println("Total GK time = $total_time")

# save GK_df with causal betas appended
CSV.write(outfile * ".txt", GK_df)

# write output
outfile = "/oak/stanford/groups/zihuai/paisa/sims/sim$(seed)_result.txt"
open(outfile, "w") do io
    println(io, "GK_power,$GK_power")
    println(io, "marginal_power,$marginal_power")
    println(io, "GK_FP,$GK_FP")
    println(io, "marginal_FP,$marginal_FP")
    println(io, "total_time,$total_time")
end

println("\nDone!")

Note: `solveblock` was compiled with `julia/1.9.0` but on Sep 10, 2024 `module load julia/1.9.0` stopped working. We either invoke julia/1.9.0 directly or we must recompile the executable (and re-solve all regions using whatever version of Julia) using the latest julia version. If we recompile, we need to check the executable can read the `EUR` data.

In [5]:
julia = "/share/software/user/open/julia/1.9.0/bin/julia" 
exe = "/oak/stanford/groups/zihuai/paisa/power_sim.jl"
LD_files = "/oak/stanford/groups/zihuai/paisa/LD_files_maf0.05"
datadir = "/oak/stanford/groups/zihuai/paisa/GLIMPSE2_maf0.05"
for sim in 11:20
    submit("$julia $exe $sim $LD_files $datadir", 1, 250, jobname="sim$sim")
end

Submitted batch job 52937525
Submitted batch job 52937575
Submitted batch job 52937576
Submitted batch job 52937577
Submitted batch job 52937578
Submitted batch job 52937579
Submitted batch job 52937580
Submitted batch job 52937581
Submitted batch job 52937582
Submitted batch job 52937583
